<a href="https://colab.research.google.com/github/Mohit-Mithra/gene-coexpression-network/blob/master/hormone-gene-classification/SMOTE_embedding_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [2]:
%cd gdrive/My\ Drive/IITM_internship
!ls

/content/gdrive/My Drive/IITM_internship
hormone_gene_graph0.txt		visualise_embeddings.ipynb
hormone_src_tgt_truegenes.txt	word_embeddings.txt
SMOTE_embedding_space.ipynb	word_vectors.json
SMOTE_oversampled_dataset.json	word_vectors.npy


In [3]:
import json

In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from collections import Counter
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
import numpy as np

In [5]:
with open('hormone_src_tgt_truegenes.txt') as json_file:
    horm_gene = json.load(json_file)

### Dataset analysis

In [6]:
print(len(horm_gene))
print(type(horm_gene))
#print(horm_gene['Adrenaline'])

59
<class 'dict'>


In [7]:
#To get count of genes for each hormone

gene_counts = {}
for hormone, genes in horm_gene.items():
    gene_counts[hormone] = [len(genes['source']), len(genes['target'])] 

### Retaining genes with embeddings

In [8]:
with open('word_embeddings.txt') as json_file:
    initial_embeddings = json.load(json_file)

print(len(initial_embeddings))

1410


In [9]:
np_load_old = np.load

np.load_new = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [10]:
extra_embeddings = np.load_new('word_vectors.npy')[()]
print(type(extra_embeddings))
#print(extra_embeddings.keys())


<class 'dict'>


In [11]:
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res

In [12]:
embeddings = Merge(initial_embeddings, extra_embeddings)
len(embeddings.keys())

1431

In [13]:
### Preparing Source Embedding dataset of SMOTE

X_dataset_src = []
y_dataset_src = []

hormone_count = 0
count_list = []

y_map_src = {}
for hormone in horm_gene.keys():
    src_num_count = len(horm_gene[hormone]['source'])   
    tar_num_count = len(horm_gene[hormone]['target'])

    if src_num_count >= 6 and tar_num_count >= 6:           ### SMOTE requires each class to have atleast 6 examples. 
        y_map_src[hormone_count] = hormone                  ### So, consider hormones that have atleast 6 source and target genes
        count_list.append(src_num_count)
        if hormone in initial_embeddings.keys():
            for src_gene in horm_gene[hormone]['source']:

                if src_gene.lower() in initial_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(initial_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

                elif src_gene.lower() in extra_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(extra_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

            hormone_count = hormone_count + 1

        elif hormone.lower() in extra_embeddings.keys():
            for src_gene in horm_gene[hormone]['source']:

                if src_gene.lower() in initial_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(initial_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

                elif src_gene.lower() in extra_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(extra_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

            hormone_count = hormone_count + 1

#print(y_dataset_src)
#print(len(X_dataset_src[0]))
print(count_list)
#print(X_dataset_src[0])
#print(X_dataset_src[1])

[8, 8, 15, 16, 32, 10, 16, 218, 21, 10, 40, 20, 20, 9, 23, 23, 12, 7, 7, 36, 24, 6]


In [14]:
### Generate new embeddings for source genes using SMOTE

counter_src = Counter(y_dataset_src)
print(counter_src)
oversample_src = SMOTE()
X_dataset_src_oversampled, y_dataset_src_oversampled = oversample_src.fit_resample(np.array(X_dataset_src), np.array(y_dataset_src))
counter = Counter(y_dataset_src_oversampled)
#print(counter)
#print(type(X_dataset_src_oversampled))
#print(len(X_dataset_src_oversampled))
#print(len(y_dataset_src_oversampled))

Counter({7: 218, 10: 40, 19: 36, 4: 32, 20: 24, 14: 23, 15: 23, 8: 21, 11: 20, 12: 20, 3: 16, 6: 16, 2: 15, 16: 12, 5: 10, 9: 10, 13: 9, 0: 8, 1: 8, 17: 7, 18: 7, 21: 6})


In [15]:
### Prepare source dataset, a dictionary of this format:  hormone_name -> 216 samples, each sample being an embedding of 200 dimensions 

source_dict = {}
for hormone, embedding in zip(y_dataset_src_oversampled, X_dataset_src_oversampled):
    if y_map_src[hormone] in source_dict:
        source_dict[y_map_src[hormone]].append(list(embedding))
    
    else:
        source_dict[y_map_src[hormone]] = [list(embedding)]

In [16]:
len(source_dict.keys())

22

In [17]:
X_dataset_tar = []
y_dataset_tar = []

hormone_count = 0
count_list = []

y_map_tar = {}
for hormone in horm_gene.keys():
    src_num_count = len(horm_gene[hormone]['source'])   
    tar_num_count = len(horm_gene[hormone]['target'])

    if src_num_count >= 6 and tar_num_count>=6:
        y_map_tar[hormone_count] = hormone
        count_list.append(tar_num_count)
        if hormone in initial_embeddings.keys():
            for tar_gene in horm_gene[hormone]['target']:

                if tar_gene.lower() in initial_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(initial_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

                elif tar_gene.lower() in extra_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(extra_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

            hormone_count = hormone_count + 1

        elif hormone.lower() in extra_embeddings.keys():
            for tar_gene in horm_gene[hormone]['target']:

                if tar_gene.lower() in initial_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(initial_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

                elif tar_gene.lower() in extra_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(extra_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

            hormone_count = hormone_count + 1

#print(y_dataset_tar)
print(len(X_dataset_tar[0]))
#print(count_list)
#print(X_dataset_tar[0])
#print(X_dataset_tar[1])

200


In [18]:
counter_tar = Counter(y_dataset_tar)
print(counter_tar)
oversample_tar = SMOTE()
X_dataset_tar_oversampled, y_dataset_tar_oversampled = oversample_tar.fit_resample(np.array(X_dataset_tar), np.array(y_dataset_tar))
counter = Counter(y_dataset_tar_oversampled)
#print(counter)
#print(type(X_dataset_tar_oversampled))
#print(len(X_dataset_tar_oversampled))
#print(len(y_dataset_tar_oversampled))

Counter({7: 300, 4: 153, 19: 150, 11: 58, 12: 58, 8: 51, 16: 51, 13: 40, 6: 37, 10: 37, 5: 36, 0: 28, 1: 28, 18: 19, 20: 19, 14: 14, 15: 12, 9: 8, 21: 8, 2: 7, 3: 6, 17: 6})


In [19]:
target_dict = {}
for hormone, embedding in zip(y_dataset_tar_oversampled, X_dataset_tar_oversampled):
    if y_map_tar[hormone] in target_dict:
        target_dict[y_map_tar[hormone]].append(list(embedding))
    
    else:
        target_dict[y_map_tar[hormone]] = [list(embedding)]

## Random Sampling

##### SMOTE generates 216 gene samples for homone source and 299 gene samples for hormone targers. Since we need equal number of source and target genes for each hormone, I randomly sampled 216 genes from target genes. 

In [20]:
for hormone in target_dict:
    random_choices = np.random.choice(len(target_dict[hormone]), len(source_dict[hormone]))
    
    target_dict[hormone] = [list(target_dict[hormone][sample]) for sample in random_choices]
    

In [21]:
len(source_dict['Adrenaline']) == len(target_dict['Adrenaline'])

True

In [22]:
dataset = {}
dataset['source'] = source_dict
dataset['target'] = target_dict

print(dataset.keys())
print('Number of hormones')
print(len(dataset['source']))
print(len(dataset['target']))
print('Number of samples for a hormone')
print(len(dataset['source']['Adrenaline']))
print(len(dataset['target']['Adrenaline']))
print('Dimension of embedding for each sample')
print(len(dataset['source']['Adrenaline'][0]))
print(len(dataset['target']['Adrenaline'][0]))

dict_keys(['source', 'target'])
Number of hormones
22
22
Number of samples for a hormone
218
218
Dimension of embedding for each sample
200
200


In [23]:
with open('SMOTE_oversampled_dataset.json', 'w') as fp:
    json.dump(dataset, fp)